# Predict habitat for native species

## Set up notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

## Import configuration and modules

In [ ]:
from remote_sensing_tools.stac_configuration import de_australia_stac_config
from remote_sensing_tools.raster_base import RasterBase, QueryParams, LoadParams
from rasterstats import zonal_stats
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from remote_sensing_tools.band_indices import calculate_indices
import xarray as xr

## Query satellite products

### Get DEA configuration and list collections

In [ ]:
config = de_australia_stac_config
config.list_collections()

### Load DEA products:
* Landsat 8 & 9 yearly geomedian

In [ ]:
query_params_ls = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2023-01-01",
    end_date="2023-12-31",
)

# Landsat 8 & 9 yearly geomedian: load only geomedian spectral bands for now
params_ls_8_9_gm = LoadParams(
    crs="EPSG:3577",
    resolution=30,
    bands=("nbart_blue", "nbart_green", "nbart_red", "nbart_nir", "nbart_swir_1", "nbart_swir_2"),
)
raster_ls_8_9_gm = RasterBase.from_stac_query(
    config=config,
    collections=["ga_ls8cls9c_gm_cyear_3"],
    query_params=query_params_ls,
    load_params=params_ls_8_9_gm,
)
ds_ls_8_9_gm= raster_ls_8_9_gm.data.isel(time=0)
ds_ls_8_9_gm

* Sentinel-2 Barest Earth and BSI

In [ ]:
query_params_s2_be = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2018-01-01",
    end_date="2018-12-31",
)

# load only bands that are used for BSI calculation
params_s2_be = LoadParams(
    crs="EPSG:3577",
    resolution=30,
    bands=("blue", "red", "nir", "swir_1"),
)
raster_s2_be = RasterBase.from_stac_query(
    config=config,
    collections=["s2_barest_earth"],
    query_params=query_params_s2_be,
    load_params=params_s2_be,
)

# calculate BSI
ds_s2_be_bsi = calculate_indices(raster_s2_be.data, ["bsi"])

# dropping original bands
ds_s2_be_bsi = ds_s2_be_bsi[["bsi"]].isel(time=0)
ds_s2_be_bsi

* SRTM DEM

In [ ]:
query_params_dem = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2014-01-01",
    end_date="2014-12-31",
)

params_dem = LoadParams(
    crs="EPSG:3577",
    resolution=30,
    bands=("dem"),
)
raster_dem= RasterBase.from_stac_query(
    config=config,
    collections=["ga_srtm_dem1sv1_0"],
    query_params=query_params_dem,
    load_params=params_dem,
)
ds_dem = raster_dem.data.isel(time=0)
ds_dem

* Multi-scale Topographic Position Index (TPI)

In [ ]:
query_params_tpi = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2018-01-01",
    end_date="2018-12-31",
)

params_tpi = LoadParams(
    crs="EPSG:3577",
    resolution=30,
    bands=("regional", "intermediate", "local"),
)
raster_tpi= RasterBase.from_stac_query(
    config=config,
    collections=["multi_scale_topographic_position"],
    query_params=query_params_tpi,
    load_params=params_tpi,
)
ds_tip = raster_tpi.data.isel(time=0)
ds_tip

* Weathering Intensity

In [ ]:
query_params_weathering = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2018-01-01",
    end_date="2018-12-31",
)

params_weathering = LoadParams(
    crs="EPSG:3577",
    resolution=30,
    bands=("intensity"),
)
raster_weathering= RasterBase.from_stac_query(
    config=config,
    collections=["weathering_intensity"],
    query_params=query_params_weathering,
    load_params=params_weathering,
)
ds_weathering = raster_weathering.data.isel(time=0)
ds_weathering

## Stack all product bands

In [ ]:
ds_all = xr.merge([ds_ls_8_9_gm, ds_s2_be_bsi, ds_dem, ds_tip, ds_weathering], compat='override')
ds_all

## Object-based classification for native species habitat prediction

### Load pre-processed training points

In [ ]:
# Frog presence points from ALA, with pseodo absence points pre-generated


### Image segmentation

### Object-level features calculation

### Model training

### Prediction